In [ ]:
import os
from datetime import UTC, datetime

In [6]:
def _get_lifecycle_tags() -> dict:
    """
    Get lifecycle tags from environment variables.

    Returns:
        Dict of tag key-value pairs (only includes tags that are set)
    """
    env_vars = {
        "argo_workflow_name": "ARGO_WORKFLOW_NAME",
        "argo_workflow_uid": "ARGO_WORKFLOW_UID",
        "git_repo": "GIT_REPO",
        "git_commit": "GIT_COMMIT",
    }

    tags = {}
    for tag_key, env_var in env_vars.items():
        value = os.getenv(env_var)
        if value:
            tags[tag_key] = value

    return tags


In [7]:
tags = _get_lifecycle_tags()
tags

{}

In [ ]:
import mlflow

client = mlflow.MlflowClient()

In [12]:
all_runs = mlflow.search_runs(search_all_experiments=True)
all_runs

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.recall_score,metrics.training_roc_auc,metrics.score,metrics.log_loss,...,tags.argo_workflow_name,tags.mlflow.source.type,tags.project,tags.mlflow.datasets,tags.estimator_name,tags.mlflow.runName,tags.estimator_class,tags.mlflow.source.git.commit,tags.mlflow.user,tags.model_family
0,6fdd147c20f642d980cf488f3900cc4d,429191795489496634,FINISHED,mlflow-artifacts:/429191795489496634/6fdd147c2...,2025-11-08 12:20:29.601000+00:00,2025-11-08 12:20:43.773000+00:00,0.972222,1.0,0.972222,0.073847,...,test,LOCAL,Wine Classification,"[{""name"":""dataset"",""hash"":""0c4d6b257f5c2dd42a0...",Pipeline,ray_wine_20251108_122029,sklearn.pipeline.Pipeline,7dc4c02bddab35b7b25d66e1770d3402596f2d83,root,LogisticRegression
1,7a1a178acd1840e787f569352391d89d,429191795489496634,FINISHED,mlflow-artifacts:/429191795489496634/7a1a178ac...,2025-11-08 12:18:32.259000+00:00,2025-11-08 12:18:47.330000+00:00,0.972222,1.0,0.972222,0.073847,...,test,LOCAL,Wine Classification,"[{""name"":""dataset"",""hash"":""0c4d6b257f5c2dd42a0...",Pipeline,ray_wine_20251108_121832,sklearn.pipeline.Pipeline,7dc4c02bddab35b7b25d66e1770d3402596f2d83,root,LogisticRegression
2,c95ba34bc76b4c099190eaa2a0b89259,429191795489496634,FINISHED,mlflow-artifacts:/429191795489496634/c95ba34bc...,2025-11-08 12:11:19.681000+00:00,2025-11-08 12:11:34.041000+00:00,0.972222,1.0,0.972222,0.073847,...,test,LOCAL,Wine Classification,"[{""name"":""dataset"",""hash"":""0c4d6b257f5c2dd42a0...",Pipeline,ray_wine_20251108_121119,sklearn.pipeline.Pipeline,7dc4c02bddab35b7b25d66e1770d3402596f2d83,root,LogisticRegression


In [28]:
workflow_name = "test"
model_name = "wine-classifier"
runs_df = mlflow.search_runs(
    search_all_experiments=True,
    filter_string=f"tags.argo_workflow_name = '{workflow_name}'",
    max_results=1,
    order_by=["start_time DESC"],
)
runs_df

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.recall_score,metrics.training_roc_auc,metrics.score,metrics.log_loss,...,tags.argo_workflow_name,tags.mlflow.source.type,tags.project,tags.mlflow.datasets,tags.estimator_name,tags.mlflow.runName,tags.estimator_class,tags.mlflow.source.git.commit,tags.mlflow.user,tags.model_family
0,6fdd147c20f642d980cf488f3900cc4d,429191795489496634,FINISHED,mlflow-artifacts:/429191795489496634/6fdd147c2...,2025-11-08 12:20:29.601000+00:00,2025-11-08 12:20:43.773000+00:00,0.972222,1.0,0.972222,0.073847,...,test,LOCAL,Wine Classification,"[{""name"":""dataset"",""hash"":""0c4d6b257f5c2dd42a0...",Pipeline,ray_wine_20251108_122029,sklearn.pipeline.Pipeline,7dc4c02bddab35b7b25d66e1770d3402596f2d83,root,LogisticRegression


In [30]:
run_id = runs_df.iloc[0]["run_id"]
run_id

'6fdd147c20f642d980cf488f3900cc4d'

In [32]:
# Step 2: Get CI model from this run
ci_versions = client.search_model_versions(
    filter_string=f"name='ci.{model_name}' and run_id='{run_id}'"
)
ci_versions

[<ModelVersion: aliases=[], creation_timestamp=1762604443694, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1762604443694, metrics=None, model_id=None, name='ci.wine-classifier', params=None, run_id='6fdd147c20f642d980cf488f3900cc4d', run_link='', source='models:/m-c53e4d87f6e6448d8846cd590a0010fa', status='READY', status_message=None, tags={}, user_id='', version='3'>]

In [43]:
ci_model = ci_versions[0]
ci_version = ci_model.version
ci_run_id = ci_model.run_id

In [44]:
ci_tags = {
    "argo_workflow_name": workflow_name,
    "argo_workflow_uid": "1234",
    "git_repo": "ai-ml-sklearn",
    "tagged_by": "ci_pipeline",
    "tagged_at": datetime.now(UTC),
}


In [45]:
for key, value in ci_tags.items():
    client.set_model_version_tag(f"ci.{model_name}", ci_version, key, value)
    print(f"   ✓ {key}")


   ✓ argo_workflow_name
   ✓ argo_workflow_uid
   ✓ git_repo
   ✓ tagged_by
   ✓ tagged_at


In [ ]:
ci_run = client.get_run(ci_run_id)
metric_name = "accuracy_score"
if metric_name not in ci_run.data.metrics:
    print(f"❌ Metric '{metric_name}' not found in CI model")
    print(f"   Available: {list(ci_run.data.metrics.keys())}")

    # Tag as rejected
    client.set_model_version_tag(
        f"ci.{model_name}", ci_version, "promotion_status", "rejected"
    )
    client.set_model_version_tag(
        f"ci.{model_name}",
        ci_version,
        "rejection_reason",
        f"Missing metric: {metric_name}",
    )

ci_value = ci_run.data.metrics[metric_name]
print(f"   CI {metric_name}: {ci_value}")

   CI accuracy_score: 0.9722222222222222
